In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

In [7]:
from june.demography.geography import Geography
from june.demography import Demography

## Initialize the world

In [25]:
%%time
geography = Geography.from_file({"super_area": ["CXB-232-C"]})

2020-06-09 07:59:24,527 - june.demography.geography - INFO - There are 2 areas and 1 super_areas in the world.
CPU times: user 14 ms, sys: 1.5 ms, total: 15.5 ms
Wall time: 15 ms
